In [1]:
import sqlite3
import pandas as pd

In [2]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None

In [3]:
conn = sqlite3.connect('parch-and-posey.db')

In [4]:
cursor = conn.cursor()
cursor.execute('''
select * from sqlite_master where type = "table";
''')
columns = [col[0] for col in cursor.description]
data = cursor.fetchall()
cursor.close()

In [5]:
pd.DataFrame(data, columns=columns)

,type,name,tbl_name,rootpage,sql
0,table,web_events,web_events,2,"CREATE TABLE web_events (\tid integer,\taccoun..."
1,table,sales_reps,sales_reps,7,"CREATE TABLE sales_reps (\tid integer,\tname b..."
2,table,region,region,222,"CREATE TABLE region (\tid integer,\tname bpchar)"
3,table,orders,orders,223,"CREATE TABLE orders (\tid integer,\taccount_id..."
4,table,accounts,accounts,583,"CREATE TABLE accounts (\tid integer,\tname bpc..."


In [6]:
pd.read_sql_query(sql='''
SELECT * FROM accounts
WHERE id BETWEEN 1500 AND 1600
''', con=conn)

,id,name,website,lat,long,primary_poc,sales_rep_id
0,1501,Intel,www.intel.com,41.031539,-74.668464,Elouise Beyers,321580
1,1511,Humana,www.humana.com,41.432337,-77.004963,Bettye Close,321590
2,1521,Disney,www.disney.com,41.878800,-74.811026,Timika Mistretta,321600
3,1531,Cisco Systems,www.cisco.com,41.201011,-76.538247,Deadra Waggener,321610
4,1541,Pfizer,www.pfizer.com,40.693260,-75.794532,Olevia Taubman,321620
5,1551,Dow Chemical,www.dow.com,40.530507,-74.666564,Lillia Ogden,321630
6,1561,Sysco,www.sysco.com,40.762524,-73.977530,Julia Laracuente,321640
7,1571,FedEx,www.fedex.com,40.752905,-73.975192,Keren Mcconn,321650
8,1581,Caterpillar,www.caterpillar.com,40.747894,-73.977387,Latia Beene,321660
9,1591,Lockheed Martin,www.lockheedmartin.com,40.754971,-73.982331,Jared Aikins,321670


In [7]:
# ONE ACCOUNT HAS NO ORDERS
pd.read_sql_query(sql='''
SELECT * FROM accounts a
LEFT JOIN orders o
ON o.account_id = a.id
WHERE o.id IS NULL
''', con=conn)

,id,name,website,lat,long,primary_poc,sales_rep_id,id,account_id,occurred_at,standard_qty,gloss_qty,poster_qty,total,standard_amt_usd,gloss_amt_usd,poster_amt_usd,total_amt_usd
0,1731,Goldman Sachs Group,www.gs.com,40.757444,-73.967309,Loris Manfredi,321690,None,None,None,None,None,None,None,None,None,None,None


In [8]:
# ALL ORDERS HAVE ACCOUNTS
pd.read_sql_query(sql='''
SELECT * FROM orders o
LEFT JOIN accounts a
ON o.account_id = a.id
WHERE a.id IS NULL
''', con=conn)

,id,account_id,occurred_at,standard_qty,gloss_qty,poster_qty,total,standard_amt_usd,gloss_amt_usd,poster_amt_usd,total_amt_usd,id,name,website,lat,long,primary_poc,sales_rep_id


In [9]:
# NO OF ORDERS ...
pd.read_sql_query(sql='''
SELECT COUNT (*) FROM orders
''', con=conn)

,COUNT (*)
0,6912


In [10]:
# ... IS THE SAME AS NUMBER OF ORDERS WITH ACCOUNT ...
pd.read_sql_query(sql='''
SELECT COUNT (*) FROM orders o
JOIN accounts a
ON o.account_id = a.id
''', con=conn)

,COUNT (*)
0,6912


In [11]:
# ... WHICH IS THIS ...
pd.read_sql_query(sql='''
SELECT COUNT(*) FROM accounts a
LEFT JOIN orders o
ON o.account_id = a.id
WHERE o.id IS NOT NULL
''', con=conn)

,COUNT(*)
0,6912


In [12]:
# ... BUT NOT THIS, WHICH IS COMPLET
pd.read_sql_query(sql='''
SELECT COUNT(*) FROM accounts a
LEFT JOIN orders o
ON o.account_id = a.id
''', con=conn)

,COUNT(*)
0,6913


In [13]:
# ACHTUNG, BABY!
pd.read_sql_query(sql='''
SELECT COUNT(o.id) FROM accounts a
LEFT JOIN orders o
ON o.account_id = a.id
''', con=conn)

,COUNT(o.id)
0,6912


In [14]:
# FULL OUTER JOIN RETUTNS 6912+1 ROWS
pd.read_sql_query(sql='''
SELECT * FROM orders o
LEFT JOIN accounts a
ON o.account_id = a.id
UNION ALL
SELECT * FROM accounts a
LEFT JOIN orders o
ON o.account_id = a.id
WHERE o.id IS NULL
''', con=conn)

,id,account_id,occurred_at,standard_qty,gloss_qty,poster_qty,total,standard_amt_usd,gloss_amt_usd,poster_amt_usd,total_amt_usd,id,name,website,lat,long,primary_poc,sales_rep_id
0,1,1001,2015-10-06 17:31:14,123.000000,22.000000,24,169,613.77,164.78,194.88,973.43,1001.0,Walmart,www.walmart.com,40.238496,-75.103297,Tamara Tuma,321500.0
1,2,1001,2015-11-05 03:34:33,190.000000,41.000000,57,288,948.10,307.09,462.84,1718.03,1001.0,Walmart,www.walmart.com,40.238496,-75.103297,Tamara Tuma,321500.0
2,3,1001,2015-12-04 04:21:55,85.000000,47.000000,0,132,424.15,352.03,0.00,776.18,1001.0,Walmart,www.walmart.com,40.238496,-75.103297,Tamara Tuma,321500.0
3,4,1001,2016-01-02 01:18:24,144.000000,32.000000,0,176,718.56,239.68,0.00,958.24,1001.0,Walmart,www.walmart.com,40.238496,-75.103297,Tamara Tuma,321500.0
4,5,1001,2016-02-01 19:27:27,108.000000,29.000000,28,165,538.92,217.21,227.36,983.49,1001.0,Walmart,www.walmart.com,40.238496,-75.103297,Tamara Tuma,321500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6908,6909,4501,2016-07-29 19:58:32,5.000000,91.000000,96,192,24.95,681.59,779.52,1486.06,4501.0,SpartanNash,www.spartannash.com,45.555651,-122.657145,Jewell Likes,321970.0
6909,6910,4501,2016-08-27 00:58:11,16.000000,94.000000,82,192,79.84,704.06,665.84,1449.74,4501.0,SpartanNash,www.spartannash.com,45.555651,-122.657145,Jewell Likes,321970.0
6910,6911,4501,2016-11-22 06:52:22,63.000000,67.000000,81,211,314.37,501.83,657.72,1473.92,4501.0,SpartanNash,www.spartannash.com,45.555651,-122.657145,Jewell Likes,321970.0
6911,6912,4501,2016-12-21 13:30:42,61.000000,150.000000,52,263,304.39,1123.50,422.24,1850.13,4501.0,SpartanNash,www.spartannash.com,45.555651,-122.657145,Jewell Likes,321970.0


In [15]:
pd.read_sql_query(sql='''
SELECT COUNT (*) FROM orders o
LEFT JOIN accounts a
ON o.account_id = a.id
UNION ALL
SELECT COUNT(*) FROM accounts a
LEFT JOIN orders o
ON o.account_id = a.id
WHERE o.id IS NULL
''', con=conn)

,COUNT (*)
0,6912
1,1


In [16]:
df = pd.read_sql_query(sql='''
SELECT COUNT (*) FROM orders o
LEFT JOIN accounts a
ON o.account_id = a.id
UNION ALL
SELECT COUNT(*) FROM accounts a
LEFT JOIN orders o
ON o.account_id = a.id
WHERE o.id IS NULL
''', con=conn)
df.loc['Total']= df.sum()
df

,COUNT (*)
0,6912
1,1
Total,6913


In [17]:
df = pd.read_sql_query(sql='''
SELECT COUNT (*) FROM orders o
LEFT JOIN accounts a
ON o.account_id = a.id
UNION ALL
SELECT COUNT(*) FROM accounts a
LEFT JOIN orders o
ON o.account_id = a.id
WHERE o.id IS NULL
''', con=conn)
df.loc['Column_Total']= df.sum(numeric_only=True, axis=0)
df.loc[:,'Row_Total'] = df.sum(numeric_only=True, axis=1)
df

,COUNT (*),Row_Total
0,6912,6912
1,1,1
Column_Total,6913,6913
